<a href="https://colab.research.google.com/github/mushamvineeth/Vineeth_INFO5731_Spring2021/blob/main/In_class_exercise_08_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The eighth in-class-exercise (20 points in total, 3/30/2021)**

The data for this exercise is from the dataset you created from assignment three. Please perform answer the following questions based on your data:

## (1) (10 points) Write a python program to extract the sentiment related terms from the corpus. You may use python package such as polyglot or external lexicon resources in the question. Rank the sentiment related terms by frequency.

In [ ]:
# Write your code here
!pip install pytreebank
import pytreebank
import sys
import os
path = os.path.join(sys.path[0],'sst_{}.txt')
data = pytreebank.load_sst('./cleaned_data')
for i in ['train', 'test']:
    with open(path.format(i), 'w') as outfile:
        for j in data[i]:
            outfile.write("__label__{}\t{}\n".format(
                j.to_labeled_lines()[0][0] + 1,
                j.to_labeled_lines()[0][1]
            ))

print(len(data['train']))


8544


In [ ]:
import nltk
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all
       | 
       | Downloading package abc to /root/nltk_data...
       |   Package abc is already up-to-date!
       | Downloading package alpino to /root/nltk_data...
       |   Package alpino is already up-to-date!
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Package biocreative_ppi is already up-to-date!
       | Downloading package brown to /root/nltk_data...
       |   Package brown is already up-to-date!
       | Downloading package brown_tei to /root/nltk_data...
       |   Package brown_tei is already up-to-date!
       | Downloading package cess_cat to /root/nltk_data...
       |   Package cess_cat is already up-to-date!
       | Down

True

In [ ]:
import pandas as pd
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
from textblob import TextBlob

data = pd.read_csv("/content/sample_data/reviwes.csv")
data = data[['Review']]
#punctuation removal
data['Review'] = data['Review'].str.replace('[^\w\s]','')
#lower case
data['Review'] = data['Review'].apply(lambda x: " ".join(x.lower() for x in x.split()))
#stop words removal
data['Review'] = data['Review'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
#tokenization
data['Review'] = data['Review'].apply(lambda x: nltk.word_tokenize(x))

freq = (data['Review']).apply(lambda x: pd.value_counts(x)).sum(axis = 0).reset_index()
freq.columns = ['words', 'tf_value']
freq['polarity_value'] = freq['words'].apply(lambda x: TextBlob(x).sentiment.polarity)

sentiment = freq.loc[freq['polarity_value'] != 0].sort_values(by='tf_value', ascending=False)
sentiment = sentiment.reset_index(drop=True)

sentiment

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,words,tf_value,polarity_value
0,good,51.0,0.700000
1,really,34.0,0.200000
2,many,30.0,0.500000
3,best,29.0,1.000000
4,great,28.0,0.800000
...,...,...,...
375,exaggerated,1.0,-0.500000
376,hot,1.0,0.250000
377,smile,1.0,0.300000
378,dishonest,1.0,-0.300000


## (2) (10 points) Compare the performance of the following tools in sentiment identification: TextBlob (https://textblob.readthedocs.io/en/dev/), VADER (https://github.com/cjhutto/vaderSentiment), TFIDF-based Support Vector Machine (SVM) (Split your data into training and testing data). Take your own annotation as the standard answers. 

Reference code: https://towardsdatascience.com/fine-grained-sentiment-analysis-in-python-part-1-2697bb111ed4

In [ ]:
# Write your code here
import pandas as pd
df=pd.read_csv("https://raw.githubusercontent.com/mushamvineeth/Vineeth_INFO5731_Spring2021/main/reviwes.csv",usecols=["Cleaned_review","sentiment"])
df=df.dropna().reset_index()

from sklearn.metrics import f1_score, accuracy_score
data_textblob = pd.read_csv("https://raw.githubusercontent.com/mushamvineeth/Vineeth_INFO5731_Spring2021/main/reviwes.csv")
data_textblob['polarity'] = data_textblob['Cleaned_review'].apply(lambda x: TextBlob(x).sentiment.polarity)
data_textblob['predicted sentiment'] = pd.cut(data_textblob['polarity'], bins=4, labels=[1, 2, 3, 4])
#Defining sentiment
def func(x):
    if x in [1, 2]:
        return 'Negative'
    if x == 3:
        return 'Neutral'
    if x == 4:
        return 'Positive'
data_textblob['predicted sentiment'] = data_textblob['predicted sentiment'].apply(lambda x: func(x))
print(data_textblob[['Document_ID', 'sentiment', 'predicted sentiment']].head(5))
acc = accuracy_score(textblob_data['sentiment'], textblob_data['predicted sentiment'])*100
f1 = f1_score(textblob_data['sentiment'], textblob_data['predicted sentiment'], average='macro')
print('accuracy of the TExtBlob:', acc)
print(' f1-score of the TextBlob:', f1)


   Document_ID sentiment predicted sentiment
0            1  Positive            Negative
1            2  Positive            Negative
2            3  Positive             Neutral
3            4  Positive            Negative
4            5  Positive             Neutral
accuracy of the TExtBlob: 42.45283018867924
 f1-score of the TextBlob: 0.3084747544241868


In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')
vader = SentimentIntensityAnalyzer()

data_vader = pd.read_csv("https://raw.githubusercontent.com/mushamvineeth/Vineeth_INFO5731_Spring2021/main/reviwes.csv")
data_vader['polarity'] = data_textblob['Cleaned_review'].apply(lambda x: vader.polarity_scores(x)['compound'])
data_vader['predicted sentiment'] = pd.cut(data_vader['polarity'], bins=4, labels=[1, 2, 3, 4])
data_vader['predicted sentiment'] = data_vader['predicted sentiment'].apply(lambda x: sentiment(x))
print(data_vader[['Document_ID', 'sentiment', 'predicted sentiment']].head(5))
accu = accuracy_score(data_vader['sentiment'], data_vader['predicted sentiment'])*100
f1 = f1_score(data_vader['sentiment'], data_vader['predicted sentiment'], average='macro')
print("Accuracy of the VADER sentiment identification:",accu)
print(' f1-score of the VADER sentiment identification :', f1)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
   Document_ID sentiment predicted sentiment
0            1  Positive            Positive
1            2  Positive            Positive
2            3  Positive            Positive
3            4  Positive            Positive
4            5  Positive            Negative
Accuracy of the VADER sentiment identification: 52.83018867924528
 f1-score of the VADER sentiment identification : 0.35142247510668556


In [ ]:
import sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

my_svm = pd.read_csv("https://raw.githubusercontent.com/mushamvineeth/Vineeth_INFO5731_Spring2021/main/reviwes.csv")
tr, te = sklearn.model_selection.train_test_split(mysvm, train_size=0.6, test_size=0.1)
my_pip = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=100, 
                                           learning_rate='optimal', tol=None))])

svm = my_pip.fit(tr['Cleaned_review'], tr['sentiment'])
te['predicted sentiment'] = svm.predict(te['Cleaned_review'])
print(te[['Document_ID', 'sentiment', 'predicted sentiment']].head(5))

acc = accuracy_score(te['sentiment'], te['predicted sentiment'])*100
f1 = f1_score(te['sentiment'], te['predicted sentiment'], average='macro')

print('Accuracy of the TFIDF-based SVM:', acc)
print('f1-score of the TFIDF-based SVM :', f1)

    Document_ID sentiment predicted sentiment
28           29  Positive            Negative
71           72   Neutral            Negative
5             6   Neutral            Positive
47           48  Positive            Positive
57           58   Neutral             Neutral
Accuracy of the TFIDF-based SVM: 54.54545454545454
f1-score of the TFIDF-based SVM : 0.5238095238095238


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [ ]:
"""from the above analysis it can be concluded that SVM has the highest f1 score next followed by 
VADER and textblob.
It can be concluded that SVM is the best tool for sentiment identification."""
